In [ ]:
%load_ext Cython

In [ ]:
%%cython

import time
import numpy as np
cimport numpy as np
import cython
from libc.string cimport memset
from scipy.linalg.cython_blas cimport dsymm, dcopy, dgemm, dgemv, daxpy, dsyrk, \
    dtrmv, dger, dnrm2, ddot, dscal
from scipy.linalg.cython_lapack cimport dpotrf, dpotrs, dpotri, dtrtrs
#from util cimport copy_transpose, copy_upper_lower

p,n,T = 10000, 20, 100

C, R = np.random.normal(size=(p,n)), np.random.normal(size=(p,))
Xm = np.random.normal(size=(n,n))

x = np.random.normal(size=(T,n))
y = x.dot(C.T)

lag_range = np.arange(n)
kl_ = np.max(lag_range)+1
ts, m_ = range(T-kl_), 0
grad = np.zeros_like(C)
a,b = np.random.choice(p, p//2, replace=False), np.random.choice(p, p//2, replace=False)
idx_grp = (a,b)
is_, js_, inj = (0,), (1,), []

CCs = [C[a,:].T.dot(C[a,:]) for a in idx_grp]
Cs = [C[a,:].copy() for a in idx_grp]
grads = [grad[a,:].copy() for a in idx_grp]

cdef void pyc_g_C_l2_vector_pair_sso(grads, int m_, Cs, double[:,:] Xm, double[:] R, 
                                     CCs, idx_grp, is_, js_, inj, yps, yfs, double[:,:] Wm):

    cdef int p = C.shape[0], n = C.shape[1]
    cdef int nn = n * n
    cdef double one = 1., zero = 0., neg1 = -1.
    cdef int one_int = 1
    cdef int pa, pb, pan, pbn
    
    cdef double[:,:] SC = np.zeros((n,n))
    cdef double[:,:] tmp_nn = np.zeros((n,n))
    cdef double[:] tmp_n = np.zeros(n)
    
    cdef double[:] Sy =  np.zeros(n)
    cdef double[:] yb, ya
    cdef double[:,:] Cb, Ca, CC, grad_a
    for i in is_:
        a = idx_grp[i]
        pa = len(a)
        ya = yfs[i]
        Ca = Cs[i]
        grad_a = grads[i]
        
        #SC = np.zeros((n,n),dtype=C.dtype)
        #Sy = np.zeros(n,dtype=C.dtype)
        memset(&SC[0,0], 0, sizeof(SC))
        memset(&Sy[0], 0, sizeof(Sy))
        for j in js_:
            b = idx_grp[j]        
            pb = len(b)            
            CC = CCs[j]
            yb = yps[j]
            Cb = Cs[j]
            
        #   SC += CCs[j] * Wm[i,j]            
            #dscal(&nn, &Wm[i,j], &CC[0,0], &one_int)
            #dgemm('N', 'N', &n, &n, &n, &zero, &SC[0,0], &n, &SC[0,0], &n, &one, &SC[0,0], &n)
        #    Sy += yp[b].dot(C[b,:]) * Wm[i,j]            
            #dgemv('T', &pb, &n, &Wm[i,j], &Cb[0,0], &pb, &yb[0], &one_int, &one, &Sy[0], &n)
            

        # tmp_nn = Xm.dot(SC)
        #dgemm('N', 'N', &n, &n, &n, &one, &Xm[0,0], &n, &SC[0,0], &n, &zero, &tmp_nn[0,0], &n) 
        # tmp_nn = tmp_nn.dot(Xm.T)
        #dgemm('N', 'T', &n, &n, &n, &one, &tmp_nn[0,0], &n, &Xm[0,0], &n, &zero, &tmp_nn[0,0], &n) 
        # grad[a,:] = grad[a,:] + C[a,:].dot(tmp_nn)
        #dgemm('N', 'N', &pa, &n, &n, &one, &Ca[0,0], &pa, &tmp_nn[0,0], &n, &one, &grad_a[0,0], &pa)        

        # tmp_n = Sy.dot(Xm.T)
        #dgemv('N', &n, &n, &one, &Xm[0,0], &n, &Sy[0], &one_int, &zero, &tmp_n[0], &one_int)                       
        # grad[a,:] = grad[a,:] - yf[a].dot(tmp_n.T)
        #dger(&n, &n, &neg1, &ya[0], &one_int, &tmp_n[0], &one_int, &tmp_nn[0,0], &n)

    #for j in js_:
    #    b = idx_grp[j]        
    #    SC = np.zeros((n,n),dtype=C.dtype) 
    #    Sy = np.zeros(n,dtype=C.dtype)
    #    for i in is_:        
    #        a = idx_grp[i]        
    #        SC += CCs[i] * Wm[i,j]
    #        Sy += yf[a].dot(C[a,:]) * Wm[i,j]
    #    grad[b,:] += C[b,:].dot( Xm.T.dot(SC).dot(Xm) ) - np.outer(yp[b], Sy.dot(Xm))
        
    #if m_ == 0:
    #    for i in inj:
    #        anb = idx_grp[i]
    #        grad[anb,:] += (R[anb]*Wm[i,i]).reshape(-1,1) * (C[anb,:].dot(Xm+Xm.T))
            
t_ = time.time()
for t in ts:
    yp, yf, Wm = y[t], y[t+m_], np.ones((len(idx_grp), len(idx_grp)))
    yps = [yp[a].copy() for a in idx_grp]
    yfs = [yf[a].copy() for a in idx_grp]    
    pyc_g_C_l2_vector_pair_sso(grad, m_, Cs, Xm, R, CCs, idx_grp, is_, js_, inj, yps, yfs, Wm)
            
print(time.time() - t_)
print(grad)

In [ ]:
from ssidid.SSID_Hankel_loss import g_C_l2_vector_pair_sso

grad = np.zeros_like(C)
t_ = time.time()
for t in ts:
    yp, yf, Wm = y[t], y[t+m_], np.ones((len(idx_grp), len(idx_grp)))
    g_C_l2_vector_pair_sso(grad, m_, C, Xm, R, CCs, idx_grp, is_, js_, inj, yp, yf, Wm)            
print(time.time() - t_)
print(grad)

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import glob, os, psutil, time
from scipy import linalg as la
from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid.utility import get_subpop_stats, gen_data
from ssidid import ObservationScheme
from subtracking import Grouse, calc_subspace_proj_error

p = 110
T = 1200

lag_range = np.arange(10)


# create subpopulations

# splitting imaging planes, 2 subpops
ns = 10

sub_pops = [np.arange(i*(p//ns), (i+1)*(p//ns)) for i in range(ns-1)] + [np.arange((ns-1)*(p//ns),p)]

obs_sweeps = 60

obs_pops = np.hstack([np.hstack((np.arange(0,len(sub_pops)),np.arange(len(sub_pops))[::-1])) for i in range(obs_sweeps)])
#obs_pops = np.hstack([np.arange(1), obs_pops, np.arange(1,len(sub_pops))])
obs_time = np.arange(1,T+1)
#obs_time = np.array([i*T//len(obs_pops) for i in range(1,len(obs_pops)+1)])


obs_scheme = ObservationScheme(p=p,T=T,
                               sub_pops=sub_pops, 
                               obs_pops=obs_pops,
                               obs_time=obs_time)
obs_scheme.gen_mask_from_scheme()

plt.figure(figsize=(8,10))
plt.subplot(3,1,1)
x = np.arange(11)
for i in range(ns+1):
    plt.plot(10*i*np.array([1,1]), np.array([0,p]), 'k')
#    plt.plot(10*i+x, 100-10*x, '--', color='b')
plt.xticks(range(10,101,10), range(1,11))
plt.axis([0,100,0,100])
#plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,1,1).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,1,1).get_xaxis().set_tick_params(direction='out')

plt.subplot(3,1,2)
x = np.arange(11)
for i in range(ns+1):
    plt.plot(10*i*np.array([1,1]), np.array([0,p]), 'k')
    if np.mod(i,2)==0:
        plt.plot(10*i+x, 100-10*x, '--', color='b')
    else:
        plt.plot(10*i+x, 10*x, '--', color='b')
plt.xticks(range(10,101,10), range(1,11))
plt.axis([0,100,0,100])
#plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,1,2).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,1,2).get_xaxis().set_tick_params(direction='out')


plt.subplot(3,1,3)
plt.imshow(obs_scheme.mask[:100,:].T, interpolation='None', aspect='auto')
plt.gray()
plt.xticks(np.arange(10,101,10)-0.5, range(1,11,1))
plt.xlabel('time [s]')
plt.ylabel('observed pixel/z-plane')
plt.subplot(3,1,3).get_yaxis().set_tick_params(direction='out')
plt.subplot(3,1,3).get_xaxis().set_tick_params(direction='out')

plt.show()

W = obs_scheme.comp_coocurrence_weights(lag_range, sso=True, idx_a=np.arange(p), idx_b=np.arange(p))

plt.figure(figsize=(10,20))
for i in range(len(lag_range)):
    plt.subplot(5,2,np.mod(i+1,10))
    plt.imshow(1/W[i], interpolation='None')
    #plt.colorbar()
    plt.gray()
    plt.ylabel('m = ' + str(i))
    plt.axis('off')
    plt.clim([0, 180])
#plt.title('variable subset co-occurence pattern')
plt.show()

tmp = np.zeros_like(W[0])
for i in range(len(W)):
    tmp += 1/W[i]
plt.imshow(tmp, interpolation='None')
plt.clim([0, 185])
plt.gray()
plt.colorbar()
plt.axis('off')
#plt.title('total co-occurences accross all lags')
plt.show()